# Banco de dados Solo Criado: Take 1

Vou fazer primeiro no notebook para ir testando, depois passo para o arquivo .py

## Roadmap

* Coletar listagem do site da prefeitura
* Transformar pdf da listagem em xmls por este site
* Transformar xmls em cvs
* Carregar cvs de listagem para o banco de dados
  * Extração;
  * Exploração;
  * Limpeza;
  * Agregação;
  * Visualização;
  * Storytelling.

## Primeira parte: Coleta e transformação dos dados

from [here](https://www.edureka.co/blog/web-scraping-with-python/): <br>
Libraries used for Web Scraping  <br>
As we know, Python is has various applications and there are different libraries for different purposes. In our further demonstration, we will be using the following libraries: <br>
<br>
**Selenium**:  Selenium is a web testing library. It is used to automate browser activities. <br>
**BeautifulSoup**: Beautiful Soup is a Python package for parsing HTML and XML documents. It creates parse trees that is helpful to extract the data easily.<br>
Pandas: Pandas is a library used for data manipulation and analysis. It is used to extract the data and store it in the desired format. 

Eu tenho que baixar com requests somente a parte que me interessa (div id="conteudo" class="outros") <br>
Ou usar o bs4 pra retirar somente essa parte do todo

O código abaixo recebe e cria um arquivo html com os dados da página de listagens.

In [24]:
#import requests
#
#url = 'http://www2.portoalegre.rs.gov.br/edificapoa/default.php?p_secao=1445'
#r = requests.get(url)
#
#open('estoques.html', 'wb').write(r.content)

O código abaixo (em andamento) serve para limpar o html e eventualmente retirar os links das listagens em pdf

In [25]:
#from bs4 import BeautifulSoup
#
#with open('estoques.html') as fp:
#    soup = BeautifulSoup(fp, 'html.parser')
#
#conteudo = soup.find(id="conteudo")
#conteudo.find_all('strong')
#conteudo.find_all('a')
#for link in conteudo.find_all('a'):
#    print(link.get('href'))

Ao longo do tempo o formato de cada entrada de listagem mudou, então o ideal seria criar uma lista com as listagens já existentes e ir adicionando com base na ultima listagem. O problema é se mudarem o formato de divulgação de novo. acho que vou focar somente na parte de banco de dados que á mais útil e confiável, por enquanto.

## Segunda parte: a criação do banco de dados

### 1. Extração

* imports

In [26]:
import pandas as pd
import os

* testando ler uma planilha de 1 listagem

In [27]:
caminho_teste = './dados/ListagemSoloCriado/XLSX/estq-max-01.xlsx'
sheet = pd.ExcelFile('./dados/ListagemSoloCriado/XLSX/estq-max-01.xlsx').sheet_names
print(sheet)
df_estq_01 = pd.read_excel(io=caminho_teste, sheet_name='Table 1', header=1)

df_estq_01.head(5)

['Table 1', 'Table 2', 'Table 3']


,MZ,UEU,QRT,Unnamed: 3
0,1,2,1,AE
1,1,2,3,AE
2,1,2,5,AE
3,1,4,3,AE
4,1,4,5,1388.5667


* lista de arquivos

In [28]:
print(os.listdir("./dados/ListagemSoloCriado/XLSX"))

['estq-max-01.xlsx', 'estq-max-02.xlsx', 'estq-max-03.xlsx', 'estq-max-04.xlsx', 'estq-max-05.xlsx', 'estq-max-06.xlsx', 'estq-max-07.xlsx', 'estq-max-08.xlsx', 'estq-max-09.xlsx', 'estq-max-10.xlsx', 'estq-max-11.xlsx', 'estq-max-12.xlsx', 'estq-max-13.xlsx', 'estq-max-14.xlsx', 'estq-max-15.xlsx', 'estq-max-16.xlsx']


Função que junta tabelas de arquivos

In [29]:
def join_sheet(file):
    """Receives a excel file path and
    returns a dataframe with his sheets
    appended. Also fix third column name."""

    sheet_list = pd.ExcelFile(file).sheet_names
    df_list = []
    for sheet in sheet_list:
        df_list.append(pd.read_excel(io=file, sheet_name=sheet, header=1))
    
    listagem = pd.concat(objs=df_list, ignore_index=True)
    
    colunas = listagem.columns.to_list()
    colunas[-1] = 'estq_max_' + file[-7:-5]
    listagem.columns = colunas

    return listagem


* Testando função join_sheet

In [30]:
estq_01 = join_sheet(caminho_teste)
estq_01

,MZ,UEU,QRT,estq_max_01
0,1,2,1,AE
1,1,2,3,AE
2,1,2,5,AE
3,1,4,3,AE
4,1,4,5,1388.5667
...,...,...,...,...
6264,8,84,21,0.0
6265,8,84,23,0.0
6266,8,84,25,0.0
6267,8,84,27,0.0


**Feito!**

exemplo de uma funÇao que cria uma lista de df com as listagens

In [31]:
def db_creator01(path):
    files = os.listdir(path)
    db_estq = []
    for table in files:
        db_estq.append(join_sheet(path + "/" + table))
    
    return db_estq

* execução da funçao

In [32]:
db_estq = db_creator01("./dados/ListagemSoloCriado/XLSX")

* 5 primeiras linhas da listagem 1

In [ ]:
db_estq[0].head(5)

* listando todas as listagens

In [ ]:
for listagem in db_estq:
    print(listagem.head(5))
    print("-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-")

**nota**: o que eu quero é uma função que:
* puxe a lista de arquivos da pasta de dados
* confere se os arquivos foram adicionados ao banco de dados(pode-se comparar i índice da lista de arquivos com o índice da lista de banco de dados)
* adiciona o arquivo ao db a partir da primeira entrada faltante
* retorna o banco de dados atualizado

In [39]:
def db_manager(path=any, database=[]):
    """Receives a path to Excel files and 
    a list containing dataframes, checks if 
    there is any file that is not in the database, 
    adds the missing files to database if there is any 
    and returns the updated database."""

    files = os.listdir(path)
    
    if len(database) == len(files): #Não há nenhum dado novo a ser adicionado
        print("O banco de dados já está atualizado.")
        return database
    elif len(database) > len(files): #Há algum problema, pois o BD possui mais entradas do que a pasta de arquivos
        print("O banco de dados possui mais entradas do que a pasta de arquivos")
        return database
    elif len(database) < len(files): #Existem novas listagens que não foram adicionadas ao BD
        print("O banco de dados está desatualizado")
        for i in range(len(database), len(files)):
            database.append(join_sheet(path + "/" + files[i]))
            print("Adicionando listagem", (i+1))

    return database
    

* Testando a função db_manager

In [48]:
db_estq = db_manager("./dados/ListagemSoloCriado/XLSX")


O banco de dados já está atualizado.


* Deu certo! <br>
Se eu chamo de novo:

In [47]:
db_estq = db_manager("./dados/ListagemSoloCriado/XLSX", db_estq)

O banco de dados já está atualizado.


* Feito!!

### 2. Exploração

Esta parte já foi feita, pois eu já conheço os dados. Cada dataframe corresponde a uma listagem, contendo as seguintes colunas:
* index
* MZ
* UEU
* QRT
* estq_max_##

### 3. Limpeza

Nesse caso não há nada o que tirar, mas é preciso adicionar a coluna MZUEUQRT, que serve como chave para muitas operações